# DEMO:Manejo de Credenciales verificables centralizado y descentralizado

Acontinuacion realizaremos una demostracion que ilustra el manejo de credenciales verificables uno centralizado y otro descentralizado.Utilizamos criptografia de clave publica para firmar y verificar estas credenciales.

Simularemos la emisión, verificación, falsificación y revocación (en el caso centralizado) de credenciales digitales.

### Importacion de librerias a trabajar: Importacion de modulos y librerias





In [ ]:
import json  # para manejar JSON
import uuid  # para generar identificadores únicos
import time  # para obtener el tiempo actual
from typing import Dict, Any, Tuple, Set, Optional # para definir tipos

# para crear widgets interactivos ,para mostrar y limpiar salidas
import ipywidgets as widgets 
from IPython.display import display, clear_output 

# para manejar criptografia
from cryptography.hazmat.primitives import hashes, serialization 
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePrivateKey, EllipticCurvePublicKey
from cryptography.exceptions import InvalidSignature

#### Constantes

In [ ]:
# Se designan los nombre a los campos usados en los diccioarios que representan credenciales

CREDENTIAL_ID = "credential_id"
USER_ID = "user_id"
SUBJECT = "subject"
CLAIM = "claim"
VALUE = "value"
TIMESTAMP = "timestamp"
MAYOR_DE_EDAD_CLAIM = "mayor_de_edad"

#### Claves y Estructuras Globales

In [ ]:
# --- Claves de la autoridad (modo centralizado) ---
authority_private_key: EllipticCurvePrivateKey = ec.generate_private_key(ec.SECP256R1())
authority_public_key: EllipticCurvePublicKey = authority_private_key.public_key()

# Lista (set) de credenciales revocadas (identificadas por su UUID)
revoked_credentials: Set[str] = set()

# Para “descentralizado”, guardamos la clave pública del usuario en este diccionario simulado de "ledger"
# ledger_users = { usuario_id_str (UUID) : public_key_object }
ledger_users: Dict[str, EllipticCurvePublicKey] = {}

#### Funciones criptográficas genericas

In [ ]:

def generate_credential_id_and_json(credential_data: Dict[str, Any]) -> Tuple[str, bytes]:
    """
    Genera un ID único para la credencial, lo añade al diccionario y serializa a JSON.
    """
    cred_id = str(uuid.uuid4())
    credential_data[CREDENTIAL_ID] = cred_id
    credential_json = json.dumps(credential_data, sort_keys=True).encode('utf-8') # sort_keys para consistencia
    return cred_id, credential_json

In [ ]:
def sign_credential_data(private_key: EllipticCurvePrivateKey, data_bytes: bytes) -> bytes:
    """
    Firma los bytes de datos proporcionados con la clave privada.
    """
    return private_key.sign(data_bytes, ec.ECDSA(hashes.SHA256()))

In [ ]:
def verify_credential_signature(public_key: EllipticCurvePublicKey, data_bytes: bytes, signature: bytes) -> bool:
    """
    Verifica la firma de los datos con la clave pública.
    Retorna True si la firma es correcta, False si no lo es.
    """
    try:
        public_key.verify(signature, data_bytes, ec.ECDSA(hashes.SHA256()))
        return True
    except InvalidSignature:
        return False

WIDGETS Y LÓGICA – MODO CENTRALIZADO

In [ ]:
# Variables de estado para centralizado
# Usamos Optional para indicar que pueden ser None inicialmente
current_central_cred_data: Optional[Dict[str, Any]] = None
current_central_cred_json: Optional[bytes] = None
current_central_signature: Optional[bytes] = None
current_central_cred_id: Optional[str] = None

In [ ]:
# Widgets para el flujo “Centralizado”
nombre_input_c = widgets.Text(description="Usuario:", placeholder="Escribe un nombre")
edad_check_c = widgets.Checkbox(description="¿Es mayor de edad?", value=True)
btn_emitir_c = widgets.Button(description="Emitir y Verificar", button_style='success')
btn_falsif_c = widgets.Button(description="Simular Falsificación", button_style='danger')
btn_revoke_c = widgets.Button(description="Revocar Credencial", button_style='warning')
output_central = widgets.Output()

In [ ]:
def emitir_y_verificar_central(btn: widgets.Button) -> None:
    """
    - Crea un objeto credencial.
    - Firma con la clave de la autoridad.
    - Verifica inmediatamente la firma y estado de revocación.
    - Muestra estado: válido/inválido/revocado.
    """
    global current_central_cred_data, current_central_cred_json, current_central_signature, current_central_cred_id
    with output_central:
        clear_output(wait=True)
        usuario = nombre_input_c.value.strip()
        es_mayor = edad_check_c.value

        if not usuario:
            print("⚠️ Debes ingresar un nombre de usuario.")
            return

        current_central_cred_data = {
            SUBJECT: usuario,
            CLAIM: MAYOR_DE_EDAD_CLAIM,
            VALUE: es_mayor,
            TIMESTAMP: time.time()
        }

        current_central_cred_id, current_central_cred_json = generate_credential_id_and_json(current_central_cred_data)
        current_central_signature = sign_credential_data(authority_private_key, current_central_cred_json)

        print("✅ [CENTRALIZADO] Credencial generada y firmada por la autoridad:")
        print(json.dumps(current_central_cred_data, indent=2))
        print(f"\n   ID de Credencial: {current_central_cred_id}")

        print("\n🔍 Verificando firma y estado de la credencial...")
        is_signature_valid = verify_credential_signature(
            authority_public_key,
            current_central_cred_json,
            current_central_signature
        )

        if not is_signature_valid:
            print("❌ ¡ERROR CRÍTICO! La firma de la credencial recién emitida es INVÁLIDA.")
            print("   Esto no debería ocurrir y podría indicar un problema en la lógica de firma/verificación.")
            return

        if current_central_cred_id in revoked_credentials:
            print(f"❌ REVOCADA: La credencial (ID: {current_central_cred_id}) ha sido revocada.")
            print("   Aunque la firma sea válida, el acceso no se permite.")
            return

        print("✅ Firma de la credencial VÁLIDA.")
        if current_central_cred_data[VALUE]:
            print("✅ ACCESO PERMITIDO: La credencial es válida y el usuario cumple los requisitos (es mayor de edad).")
        else:
            print("❌ ACCESO DENEGADO: La credencial es válida, pero el usuario NO cumple los requisitos (no es mayor de edad).")


In [ ]:
def falsificar_credencial_central(btn: widgets.Button) -> None:
    """
    - Simula una alteración de la credencial centralizada actual.
    - Intenta verificar la firma con el JSON modificado; debe FALLAR.
    """
    with output_central:
        clear_output(wait=True)
        if current_central_cred_data is None or current_central_signature is None:
            print("⚠️ Primero debes generar una credencial (Emitir y Verificar).")
            return

        # Simular modificación maliciosa
        cred_falsa_data = current_central_cred_data.copy()
        cred_falsa_data[VALUE] = not current_central_cred_data[VALUE] # Invertimos el valor

        # Es importante recalcular el JSON porque el orden de las claves podría cambiar
        # o si se añade/quita el cred_id_central, aunque aquí lo mantenemos igual
        # por simplicidad de la falsificación.
        # Para una falsificación "perfecta" que intente reutilizar el ID, el falsificador
        # tendría que asegurarse que el JSON es idéntico excepto por el valor.
        # Aquí, simplemente generamos el JSON a partir del diccionario modificado.
        _ , cred_falsa_json = generate_credential_id_and_json(cred_falsa_data)
        # Pero usamos la firma ORIGINAL

        print("🛑 [CENTRALIZADO] Intentando verificar una credencial falsificada:")
        print("   Contenido Original:")
        print(json.dumps(current_central_cred_data, indent=2))
        print("   Contenido Falsificado (solo 'value' cambiado):")
        print(json.dumps(cred_falsa_data, indent=2))


        is_falsa_signature_valid = verify_credential_signature(
            authority_public_key,
            cred_falsa_json, # Usamos el JSON de la credencial alterada
            current_central_signature # Pero la firma de la original
        )

        if is_falsa_signature_valid:
            print("🚨 ¡FALLO DE SEGURIDAD! La verificación de la credencial falsificada fue EXITOSA.")
            print("   Esto NO debería pasar y sugiere un error en la lógica o una debilidad.")
        else:
            print("✅ DETECCIÓN CORRECTA: La firma de la credencial falsificada es INVÁLIDA, como se esperaba.")


In [ ]:
def revocar_credencial_central(btn: widgets.Button) -> None:
    """
    - Revoca la credencial centralizada actual añadiendo su ID a `revoked_credentials`.
    """
    with output_central:
        clear_output(wait=True)
        if current_central_cred_id is None:
            print("⚠️ Primero debes generar una credencial para poder revocarla.")
            return

        if current_central_cred_id in revoked_credentials:
            print(f"ℹ️ La credencial (ID: {current_central_cred_id}) ya estaba revocada.")
        else:
            revoked_credentials.add(current_central_cred_id)
            print(f"🛑 REVOCADA: La credencial (ID: {current_central_cred_id}) ha sido añadida a la lista de revocación.")

        print("\n   Cualquier intento futuro de verificar esta credencial indicará que está revocada,")
        print("   incluso si la firma criptográfica sigue siendo técnicamente válida.")

In [ ]:
# Conectar botones
btn_emitir_c.on_click(emitir_y_verificar_central)
btn_falsif_c.on_click(falsificar_credencial_central)
btn_revoke_c.on_click(revocar_credencial_central)

In [ ]:
# Contenedor visual para centralizado
central_box = widgets.VBox([
    widgets.HTML("<h2>MODO CENTRALIZADO (Autoridad Emisora Única)</h2>"),
    nombre_input_c,
    edad_check_c,
    widgets.HBox([btn_emitir_c, btn_falsif_c, btn_revoke_c]),
    output_central
])

WIDGETS Y LÓGICA – MODO DESCENTRALIZADO

In [ ]:
# Variables de estado para descentralizado
current_user_private_key: Optional[EllipticCurvePrivateKey] = None
current_user_public_key: Optional[EllipticCurvePublicKey] = None
current_user_id_str: Optional[str] = None # UUID del usuario en nuestro ledger simulado

current_decent_cred_data: Optional[Dict[str, Any]] = None
current_decent_cred_json: Optional[bytes] = None
current_decent_signature: Optional[bytes] = None
# current_decent_cred_id no es tan crucial aquí como en centralizado para revocación,
# pero se sigue generando y guardando en la credencial.

In [ ]:
# Widgets para el flujo “Descentralizado”
btn_gen_keys = widgets.Button(description="1. Generar Mi Identidad (Claves)", button_style='info')
lbl_user_info = widgets.HTML("")
nombre_input_d = widgets.Text(description="Atributo 'Sujeto':", placeholder="Escribe un nombre (opcional)")
edad_check_d = widgets.Checkbox(description="Atributo '¿Mayor de edad?':", value=True)
btn_sign_d = widgets.Button(description="2. Firmar Mi Credencial", button_style='success')
btn_verify_d = widgets.Button(description="3. Verificar Credencial (Como Verificador)", button_style='primary')
btn_falsif_d = widgets.Button(description="4. Simular Falsificación", button_style='danger')
output_decent = widgets.Output()

In [ ]:
def generate_user_keys_decentralized(btn: widgets.Button) -> None:
    """
    - Genera un par de claves para el usuario (self-sovereign).
    - Simula el registro de la clave pública en un "ledger".
    """
    global current_user_private_key, current_user_public_key, current_user_id_str
    with output_decent:
        clear_output(wait=True)
        current_user_private_key = ec.generate_private_key(ec.SECP256R1())
        current_user_public_key = current_user_private_key.public_key()
        current_user_id_str = str(uuid.uuid4()) # Este ID es para nuestro "ledger" simulado

        ledger_users[current_user_id_str] = current_user_public_key

        pem = current_user_public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')

        lbl_user_info.value = (
            f"<b>Identidad Generada:</b><br>"
            f"<b>ID de Usuario (en Ledger):</b> {current_user_id_str}<br>"
            f"<b>Clave Pública (PEM format):</b><br><pre style='font-size:10px; word-wrap:break-word;'>{pem}</pre>"
        )
        print("✅ Identidad (par de claves) generada y clave pública 'registrada' en el ledger.")
        print("   Ahora puedes crear y firmar tu propia credencial.")

def sign_credential_decentralized(btn: widgets.Button) -> None:
    """
    - El usuario firma su propia credencial con su clave privada.
    - La credencial incluye el ID del usuario para que el verificador pueda buscar su clave pública.
    """
    global current_decent_cred_data, current_decent_cred_json, current_decent_signature
    with output_decent:
        clear_output(wait=True)
        if current_user_private_key is None or current_user_id_str is None:
            print("⚠️ Primero debes generar tu identidad (par de claves) en el paso 1.")
            return

        usuario_declarado = nombre_input_d.value.strip()
        es_mayor_declarado = edad_check_d.value

        current_decent_cred_data = {
            USER_ID: current_user_id_str, # Importante: para que el verificador encuentre la clave pública
            SUBJECT: usuario_declarado,   # El usuario decide qué sujeto incluir
            CLAIM: MAYOR_DE_EDAD_CLAIM,
            VALUE: es_mayor_declarado,
            TIMESTAMP: time.time()
        }

        _ , current_decent_cred_json = generate_credential_id_and_json(current_decent_cred_data)
        current_decent_signature = sign_credential_data(current_user_private_key, current_decent_cred_json)

        # Extraer el ID de la credencial del diccionario después de que generate_credential_id_and_json lo añada
        cred_id_display = current_decent_cred_data.get(CREDENTIAL_ID, "N/A")


        print("✅ [DESCENTRALIZADO] Credencial auto-firmada por el usuario:")
        print(json.dumps(current_decent_cred_data, indent=2))
        print(f"\n   ID de Credencial: {cred_id_display}")
        print(f"   Firma (hex):\n   {current_decent_signature.hex()}")
        print("\n➡️ A continuación, un verificador puede comprobar esta credencial (paso 3).")